In [1]:
import pandas as pd
import re

In [2]:
# Loading population df
pop_df = pd.read_csv('data/PEP_2017_PEPAGESEX.csv', skiprows=1)

In [3]:
# Loading cancer df
cancer_df = pd.read_csv('data/cancer_deaths.csv', skiprows=4, skipfooter=14,
                        engine='python')

In [4]:
# drop all April estimate columns

In [5]:
# Columns to keep
cols_to_keep = ['Geography']
for col in pop_df.columns:
    sep_words = col.split('-')
    if len(sep_words) == 4 and 'Total' in col and 'Both' in col:
        ages = sep_words[3]
        min_age = min(map(int, re.findall(r'\d+', ages)))
        if min_age >= 50:
            cols_to_keep.append(col)
    elif 'Median' in col and 'Both' in col:
        cols_to_keep.append(col)
    elif len(sep_words) == 3 and 'Total' in col and 'Both' in col:
        cols_to_keep.append(col)


In [6]:
pop_over50 = pop_df[cols_to_keep]

In [7]:
cols_to_drop = [col for col in pop_over50.columns if col.startswith('April') 
                 or '2011' in col or '2012' in col or '2013' in col
                or '2015' in col or '2016' in col]

In [8]:
pop_ready = pop_over50.drop(cols_to_drop, axis='columns')


In [9]:
def update_name(words):
    sep_words = words.split('-')
    if len(sep_words) == 3:
        year = sep_words[1]
        if 'Median' in words:
            new_name = 'median_age_' + year
        else:
            new_name = 'pop' + year + 'all'
    elif len(sep_words) == 4:
        year = sep_words[1]
        ages = list(map(int, re.findall(r'\d+', sep_words[-1])))
        if len(ages) == 2:
            new_name = 'pop{0}_{1}to{2}'.format(year, ages[0], ages[1])
        else:
            new_name = 'pop{0}_{1}over'.format(year, ages[0])
    else:
        new_name = 'County'
    return new_name
        

In [10]:
# Updating pop_ready names to

pop_ready.rename(update_name, axis='columns', inplace=True)

In [11]:
# Now we have to make the names of the county match
def remove_tn(county):
    county = county.split(',')
    only_county = county[0]
    return only_county

pop_ready['County'] = pop_ready['County'].apply(remove_tn)

# We drop the first two rows of the dataframe.
cancer_df.drop([0, 1], axis=0, inplace=True)

## Use an outer join to merge pop_df and cancer data to a DataFrame called pop_and_cancer

In [12]:
# Sorting by county column to have the same order
pop_ready.sort_values(by=['County'], inplace=True)
cancer_df.sort_values(by=['County'], inplace=True)

# reset index after deleting rows and sorting
pop_ready.reset_index(drop=True, inplace=True)
cancer_df.reset_index(drop=True, inplace=True)

pop_and_cancer = pd.concat([pop_ready, cancer_df], join='outer', axis='columns')

# We drop one of the repeated County columns that is a result of joining the data frames
pop_and_cancer = pop_and_cancer.iloc[:, ~pop_and_cancer.columns.duplicated()]

## Use the pandas series value_counts() method to create a dictionary called hospice_per_county. The key for this dict will be the county name and the value will be the count of hospice facilities in that county. Populate the dictionary for all TN counties by adding the counties with 0 hospice facilities to hospice_per_county.

In [13]:
# Loading hospice data
hospice_df = pd.read_csv('data/Hospice_General_Information.csv')

# Keep only the rows where facilities are in TN.
hospice_tn = hospice_df[hospice_df['State'] == 'TN']

# Keep only the Facility Name, Address Line 1, City, State, County Name, CMS Region, 
# Ownership Type, and Cerfification Date columns. 

hospice_ready = hospice_tn.drop(['CMS Certification Number (CCN)', 'Address Line 2', 
                 'Zip Code', 'PhoneNumber'], axis=1)

# Make all column names lower case and without spaces.
def lower_nospace(words):
    new_word = words.replace(" ", "")
    new_word = new_word.lower()
    return new_word

hospice_ready.rename(lower_nospace, axis='columns', inplace=True)

In [14]:
# Now we have to make the names of the county match
def add_county(county):
    county = county + ' County'
    return county

hospice_ready['countyname'] = hospice_ready['countyname'].apply(add_county)

In [15]:
hospice_county = hospice_ready['countyname'].value_counts().to_dict()
for county in pop_and_cancer['County'].values:
    if county in list(hospice_county.keys()):
        pass
    else:
        hospice_county[county] = 0

In [16]:
pop_and_cancer['hospice_per_county']= pop_and_cancer['County'].map(hospice_county)

In [18]:
pop_and_cancer.isnull().values.any()


False

In [19]:
pop_and_cancer

,County,pop 2010 all,pop 2014 all,pop 2017 all,pop 2010 _50to54,pop 2014 _50to54,pop 2017 _50to54,pop 2010 _55to59,pop 2014 _55to59,pop 2017 _55to59,...,Met Healthy People Objective of 161.4?,"Age-Adjusted Death Rate - deaths per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend in Death Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,hospice_per_county
0,Anderson County,75112,75131,76257,5850,5690,5296,5577,5689,5643,...,No,180.6,169.1,192.7,195,falling,-0.7,-1.2,-0.2,0
1,Bedford County,45086,46259,48117,3043,3207,3239,2646,2938,3143,...,No,182.7,166.1,200.6,92,falling,-0.7,-1.4,-0.1,0
2,Benton County,16511,16176,15986,1259,1204,1179,1224,1239,1206,...,No,233.9,207.1,263.6,59,stable,0.3,-0.4,1.1,0
3,Bledsoe County,12882,14419,14717,1005,1216,1185,903,1100,1197,...,No,170.0,143.5,200.6,31,falling,-1.3,-2.3,-0.3,0
4,Blount County,123213,125827,129929,9152,9658,9522,8657,9094,9615,...,No,168.2,159.4,177.4,283,falling,-0.9,-1.3,-0.4,1
5,Bradley County,99096,102816,105560,6967,7306,7216,6177,6774,7199,...,No,183.0,172.2,194.3,220,falling,-1.0,-1.4,-0.5,2
6,Campbell County,40735,39895,39648,2883,2863,2866,2864,2810,2719,...,No,219.2,201.8,237.7,124,stable,-0.4,-0.8,0.0,0
7,Cannon County,13801,13614,14216,1093,1080,1081,930,994,1059,...,No,202.4,174.2,234.3,38,stable,-0.2,-1.1,0.7,0
8,Carroll County,28448,28358,27860,1966,2044,1959,1945,1944,1968,...,No,221.8,201.2,244.1,89,stable,-0.2,-0.7,0.2,2
9,Carter County,57340,56270,56488,4311,4195,4145,4099,4273,4124,...,No,176.2,163.4,189.9,145,falling,-1.0,-1.6,-0.4,1
